## 0. 개발 환경 구성

#### 1) pgadmin4 다운로드

> https://www.enterprisedb.com/downloads/postgres-postgresql-downloads

> https://losskatsu.github.io/it-infra/postgresql-win/#%ED%94%84%EB%A1%A0%ED%8A%B8%EC%97%94%EB%93%9C

#### 1) 가상 환경 생성

```
python -m venv lecture

call lecture/Scripts/activate
```

In [1]:
!python -m venv lecture


#### 2) 필요 패키지 설치

```
pip install -r requirements.txt
```

---

In [2]:
!pip install -r requirements.txt

  Using cached psycopg2-2.9.9-cp311-cp311-macosx_11_0_arm64.whl
  Using cached pandas-2.1.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached numpy-1.26.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (115 kB)
  Using cached ipykernel-6.25.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached SQLAlchemy-2.0.22-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.4 kB)
  Using cached PyMySQL-1.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached pyarrow-14.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
Using cached pandas-2.1.1-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
Using cached numpy-1.26.1-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Using cached ipykernel-6.25.2-py3-none-any.whl (154 kB)
Using cached SQLAlchemy-2.0.22-cp311-cp311-macosx_11_0_arm64.whl (2.0 MB)
Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Using cached PyMySQL-1.1.0-py3-none-any.whl (44 kB)
Using cached pyarrow-14.0.0-cp311-cp311-macosx_11_0_arm64.whl (24.0 MB)
  Attempting uninstall: num

## 1. Python을 통한 DB Connection

### 1-1 mysql 라이브러리(psycopg2)를 활용한 Connection

##### 1) Connection

In [17]:
import psycopg2

# database connection 생성
db = psycopg2.connect(host='localhost', dbname='postgres', user='postgres', password='12341234', port=5432)

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor = db.cursor()

##### 2) CRUD Execution

In [2]:
create_query = "CREATE TABLE lecture (id SERIAL PRIMARY KEY, name VARCHAR(32), year INT, gender VARCHAR(10), count INT);"

insert_query = "INSERT INTO lecture VALUES(1, 'Ann', 2023, 'M', 1100);"

update_query = """
            UPDATE lecture 
            SET id = 1,
                name = 'HYUNSOO',
                year = '1991',
                gender = 'M',
                count = '1000'
            WHERE id = 1;
            """
delete_query = "DELETE FROM lecture WHERE id = 2001;"

drop_table = 'DROP TABLE lecture'

#### Connection Pool

- 일정량의 Connection 객체를 미리 만들어서 pool에 저장했다가, 클라이언트 요청이 오면 Connection 객체를 빌려주고 해당 객체의 임무가 완료되면 다시 Connection 객체를 반납 받아서 pool에 저장하는 프로그래밍 기법이다.
- 따라서 자원을 사용하면 반드시 반환해줘야하는데 이를 위해 DB 연결시 파이썬 with 구문을 사용한다.
- 너무 많은 DB 접근이 발생할 경우에는 커넥션은 한정되어 있기 때문에 쓸 수 있는 커넥션이 발납될 때까지 기다려야 한다. 너무 많은 커넥션을 생성할 시에는 커넥션 또한 객체이므로 많은 메모리를 차지하게 되고, 프로그램의 성능을 떨어뜨리는 원인이 된다.
- 즉, 커넥션 풀을 크게 설정하면 메모리 소모가 큰 대신 많은 사용자가 대기 시간이 줄어 들고, 반대로 커넥션 풀을 작게 설정하면 그 만큼 대기 시간이 길어진다. 따라서 사용량에 따라 적정량의 커넥션 객체를 생성해 두어야 한다.

In [16]:
cursor.close()

##### 3) Pandas를 활용한 Bulk Insert

In [2]:
import pandas as pd


In [12]:
df = pd.read_csv('../data/national_names.csv')

In [13]:
df.iloc[1]

id           2
name      Anna
year      1880
gender       F
count     2604
Name: 1, dtype: object

In [ ]:

for i in range(len(df)):
    datas = tuple(df.iloc[i])
    print(datas)
    query = f"INSERT INTO lecture VALUES{datas};"
    cursor.execute(query)

db.commit()

In [18]:
cursor.execute("SELECT * FROM lecture limit 5")
cursor.fetchall() #select쿼리의 결과를 가져옴

[(1, 'Mary', 1880, 'F', 7065),
 (2, 'Anna', 1880, 'F', 2604),
 (3, 'Emma', 1880, 'F', 2003),
 (4, 'Elizabeth', 1880, 'F', 1939),
 (5, 'Minnie', 1880, 'F', 1746)]

### 1-2. ORM 라이브러리(sqlalchemy)를 활용한 Connection

#### ORM(Object Relational Mapping) 이란?
- Python 객체와 관계형 DB의 data를 매핑해주는 것
- 장점
    - Query가 아닌 코드로 데이터를 조작할 수 있어 개발자가 프로그래밍에 더 집중할 수 있도록 도와준다.
    - 재사용 및 유지보수의 편리성이 증가한다.
    - DBMS에 대한 종속성이 줄어든다.

- 단점
  - 완벽한 ORM으로만 서비스를 구현하기 어렵다.
  - 프로시저가 많은 시스템에선 ORM의 객체 지향적인 장점을 활용하기 어렵다.

##### 1) Connection 및 Session

In [18]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 접속이 끝나더라도 계속해서 연결 상태를 유지시키기 위해 sessionmaker를 통해 세션을 생성

engine_name = 'postgresql'
user_id = 'postgres'
user_pw = '12341234'
host = 'localhost'
port = '5432'
db_name = 'postgres'

db = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{port}/{db_name}')

Session = sessionmaker(db)
session = Session()

##### 2) 매핑 테이블 생성

In [19]:
from sqlalchemy import Column, INTEGER, VARCHAR
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class lecture(Base):
    __tablename__ = 'lecture'
    
    id = Column('id', INTEGER, primary_key=True, autoincrement=True) # autoincrement : 자동으로 1씩 증가
    name = Column('name', VARCHAR(10), nullable=False)
    year = Column('year', INTEGER, nullable=False)
    gender = Column('gender', VARCHAR(10), nullable=False)
    count = Column('count', INTEGER, nullable=False)

# Create
Base.metadata.create_all(db) # lecture 클래스에 정의된 메타데이터를 바탕으로 실제 데이터베이스 테이블을 생성

# Drop
# Sample.__table__.drop(db)

##### 3) ORM을 활용한 CRUD

In [20]:
# 1) SELECT ONE

res = session.query(lecture).first()

print(res.id, res.name, res.year, res.gender, res.count)

1 Mary 1880 F 7065


In [24]:
# 1) SELECT ALL

res = session.query(lecture).all()

for i in res:
    print(i.id, i.name, i.year, i.gender, i.count)

1 Mary 1880 F 7065
2 Anna 1880 F 2604
3 Emma 1880 F 2003
4 Elizabeth 1880 F 1939
5 Minnie 1880 F 1746
6 Margaret 1880 F 1578
7 Ida 1880 F 1472
8 Alice 1880 F 1414
9 Bertha 1880 F 1320
10 Sarah 1880 F 1288
11 Annie 1880 F 1258
12 Clara 1880 F 1226
13 Ella 1880 F 1156
14 Florence 1880 F 1063
15 Cora 1880 F 1045
16 Martha 1880 F 1040
17 Laura 1880 F 1012
18 Nellie 1880 F 995
19 Grace 1880 F 982
20 Carrie 1880 F 949
21 Maude 1880 F 858
22 Mabel 1880 F 808
23 Bessie 1880 F 796
24 Jennie 1880 F 793
25 Gertrude 1880 F 787
26 Julia 1880 F 783
27 Hattie 1880 F 769
28 Edith 1880 F 768
29 Mattie 1880 F 704
30 Rose 1880 F 700
31 Catherine 1880 F 688
32 Lillian 1880 F 672
33 Ada 1880 F 652
34 Lillie 1880 F 647
35 Helen 1880 F 636
36 Jessie 1880 F 635
37 Louise 1880 F 635
38 Ethel 1880 F 633
39 Lula 1880 F 621
40 Myrtle 1880 F 615
41 Eva 1880 F 614
42 Frances 1880 F 605
43 Lena 1880 F 603
44 Lucy 1880 F 590
45 Edna 1880 F 588
46 Maggie 1880 F 582
47 Pearl 1880 F 569
48 Daisy 1880 F 564
49 Fannie 188

In [22]:
# 2) INSERT

data1 = lecture(id=2002, name='jiyun', year=2000, gender='F', count=1234)
session.add(data1)
session.commit()

In [23]:
# 3) UPDATE

session.query(lecture).filter(lecture.name == 'ung').update({'gender':'M', 'count':1234})
session.commit()

In [60]:
# 4) DELETE

session.query(lecture).filter(lecture.id == 2).delete()
session.commit()

## 2. dotenv 라이브러리를 활용한 민감 정보 관리

- 환경 변수에 대한 관리를 효과적이고 안전하게 할 수 있도록 도와주는 python 라이브러리
- DB정보, 비밀번호, API KEY 등 외부에 공유되거나 Git에 올라가면 안되는 값들을 하드코딩 하지 않고 사용이 가능.

    ```
    pip install python-dotenv
    ```

    > https://velog.io/@newnew_daddy/python-dotenv

##### 1) 기본 기능 사용

In [25]:
import dotenv

# .env 경로 찾기
env_path = dotenv.find_dotenv()
print(env_path)

# .env 파일 load
dotenv.load_dotenv(env_path)

# .env 파일 변수값 출력
dotenv.dotenv_values(env_path)

# .env 파일 변수값 변경
# dotenv.set_key(env_path, '변경할 KEY', '변경할 VALUE')

/Users/bearjang/Downloads/DataEngineering/day1/.env


OrderedDict([('POSTGRES_ENGINE', 'postgresql'),
             ('POSTGRES_HOST', 'localhost'),
             ('POSTGRES_DB_1', 'postgres'),
             ('POSTGRES_USER', 'postgres'),
             ('POSTGRES_PASSWORD', '12341234'),
             ('POSTGRES_PORT', '5432')])

##### 2) .env 파일 작성

In [26]:
POSTGRES_ENGINE = 'postgresql'
POSTGRES_HOST ='localhost'
POSTGRES_DB_1 = 'postgres'
POSTGRES_USER ='postgres'
POSTGRES_PASSWORD = '12341234'
POSTGRES_PORT = '5432'
 

##### 3) 값들 Load해오기

In [27]:
import os

engine = os.environ.get('POSTGRES_ENGINE')
host = os.environ.get('POSTGRES_HOST')
dbname = os.environ.get('POSTGRES_DB_1')
user = os.environ.get('POSTGRES_USER')
password = os.environ.get('POSTGRES_PASSWORD')
port = os.environ.get('POSTGRES_PORT')


In [28]:
import psycopg2

# database connection 생성
db = psycopg2.connect(host=host, dbname=dbname, user=user, password=password, port=port)

# 커서 생성 > 특정 SQL 문장을 처리한 결과를 담고 있는 영역을 가리키는 일종의 포인터 / 쿼리문에 의해서 반환되는 결과값들을 저장하는 메모리공간
cursor = db.cursor()

## 3. Python Class를 사용한 DB 접속 관리

##### 1) Class 작성

In [29]:
class DBconnector:

    def __init__(self, host, dbname, user, password, port):
        self.host = host
        self.dbname = dbname
        self.user = user
        self.password = password
        self.port = port
        self.connect = self.postgres_connect()

    
    def __enter__(self):
        self.connect
        print('접속')
        return self


    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print('종료')

    def postgres_connect(self):
        self.conn = psycopg2.connect(host = self.host,
                        dbname = self.dbname,
                        user = self.user,
                        password = self.password,
                        port = self.port)
        
        return self

In [30]:
(host, dbname, user, password, port)

('localhost', 'postgres', 'postgres', '12341234', '5432')

In [31]:
db_object = DBconnector(host, dbname, user, password, port)
db_conn = db_object.postgres_connect().conn

cursor = db_conn.cursor()

In [32]:
db_object = DBconnector(host, dbname, user, password, port)
# db_conn = db_object.postgres_connect().conn

with db_object as connected:
    db_conn = connected.conn
    cursor = db_conn.cursor()
    
    cursor.execute("SELECT * FROM lecture LIMIT 5")
    print(cursor.fetchall())

접속
[(1, 'Mary', 1880, 'F', 7065), (2, 'Anna', 1880, 'F', 2604), (3, 'Emma', 1880, 'F', 2003), (4, 'Elizabeth', 1880, 'F', 1939), (5, 'Minnie', 1880, 'F', 1746)]
종료


##### 3) connection parameter 합치기

In [36]:
class DBconnector:

    def __init__(self, host, dbname, user, password, port):

        self.conn_params = dict(
            host = host,
            dbname = dbname,
            user = user,
            password = password,
            port = port
        )
            
        # self.host = host
        # self.dbname = db_name
        # self.user = user
        # self.password = password
        # self.port = port
        self.connect = self.postgres_connect()

    
    def __enter__(self):
        self.connect
        print('접속')
        return self


    def __exit__(self, exc_type, exc_value, traceback):
        self.conn.close()
        print('종료')

    def postgres_connect(self):
        self.conn = psycopg2.connect(**self.conn_params)
        
        return self